## Markov switching dynamic regression models

This notebook provides an example of the use of Markov switching models in statsmodels to estimate dynamic regression models with changes in regime. It follows the examples in the Stata Markov switching documentation, which can be found at http://www.stata.com/manuals14/tsmswitch.pdf.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

# NBER recessions
from pandas_datareader.data import DataReader
from datetime import datetime

usrec = DataReader(
    "USREC", "fred", start=datetime(1947, 1, 1), end=datetime(2013, 4, 1)
)

### Federal funds rate with switching intercept

The first example models the federal funds rate as noise around a constant intercept, but where the intercept changes during different regimes. The model is simply:

$$r_t = \mu_{S_t} + \varepsilon_t \qquad \varepsilon_t \sim N(0, \sigma^2)$$

where $S_t \in \{0, 1\}$, and the regime transitions according to

$$ P(S_t = s_t | S_{t-1} = s_{t-1}) =
\begin{bmatrix}
p_{00} & p_{10} \\
1 - p_{00} & 1 - p_{10}
\end{bmatrix}
$$

We will estimate the parameters of this model by maximum likelihood: $p_{00}, p_{10}, \mu_0, \mu_1, \sigma^2$.

The data used in this example can be found at https://www.stata-press.com/data/r14/usmacro.

In [ ]:
# Get the federal funds rate data
fedfunds = [1.03, 0.99, 1.34, 1.5, 1.94, 2.36, 2.48, 2.69, 2.81, 2.93, 2.93,
            3.0, 3.23, 3.25, 1.86, 0.94, 1.32, 2.16, 2.57, 3.08, 3.58, 3.99,
            3.93, 3.7, 2.94, 2.3, 2.0, 1.73, 1.68, 2.4, 2.46, 2.61, 2.85,
            2.92, 2.97, 2.96, 3.33, 3.45, 3.46, 3.49, 3.46, 3.58, 3.97, 4.08,
            4.07, 4.17, 4.56, 4.91, 5.41, 5.56, 4.82, 3.99, 3.89, 4.17, 4.79,
            5.98, 5.94, 5.92, 6.57, 8.33, 8.98, 8.94, 8.57, 7.88, 6.7, 5.57,
            3.86, 4.56, 5.47, 4.75, 3.54, 4.3, 4.74, 5.14, 6.54, 7.82, 10.56,
            10.0, 9.32, 11.25, 12.09, 9.35, 6.3, 5.42, 6.16, 5.41, 4.83, 5.2,
            5.28, 4.87, 4.66, 5.16, 5.82, 6.51, 6.76, 7.28, 8.1, 9.58, 10.07,
            10.18, 10.95, 13.58, 15.05, 12.69, 9.84, 15.85, 16.57, 17.78,
            17.58, 13.59, 14.23, 14.51, 11.01, 9.29, 8.65, 8.8, 9.46, 9.43,
            9.69, 10.56, 11.39, 9.27, 8.48, 7.92, 7.9, 8.1, 7.83, 6.92, 6.21,
            6.27, 6.22, 6.65, 6.84, 6.92, 6.66, 7.16, 7.98, 8.47, 9.44, 9.73,
            9.08, 8.61, 8.25, 8.24, 8.16, 7.74, 6.43, 5.86, 5.64, 4.82, 4.02,
            3.77, 3.26, 3.04, 3.04, 3.0, 3.06, 2.99, 3.21, 3.94, 4.49, 5.17,
            5.81, 6.02, 5.8, 5.72, 5.36, 5.24, 5.31, 5.28, 5.28, 5.52, 5.53,
            5.51, 5.52, 5.5, 5.53, 4.86, 4.73, 4.75, 5.09, 5.31, 5.68, 6.27,
            6.52, 6.47, 5.59, 4.33, 3.5, 2.13, 1.73, 1.75, 1.74, 1.44, 1.25,
            1.25, 1.02, 1.0, 1.0, 1.01, 1.43, 1.95, 2.47, 2.94, 3.46, 3.98,
            4.46, 4.91, 5.25, 5.25, 5.26, 5.25, 5.07, 4.5, 3.18, 2.09, 1.94,
            0.51, 0.18, 0.18, 0.16, 0.12, 0.13, 0.19, 0.19, 0.19]

dta_fedfunds = pd.Series(
    fedfunds, index=pd.date_range("1954-07-01", "2010-10-01", freq="QS")
)

# Plot the data
dta_fedfunds.plot(title="Federal funds rate", figsize=(12, 3))

# Fit the model
# (a switching mean is the default of the MarkovRegession model)
mod_fedfunds = sm.tsa.MarkovRegression(dta_fedfunds, k_regimes=2)
res_fedfunds = mod_fedfunds.fit()

In [ ]:
res_fedfunds.summary()

From the summary output, the mean federal funds rate in the first regime (the "low regime") is estimated to be $3.7$ whereas in the "high regime" it is $9.6$. Below we plot the smoothed probabilities of being in the high regime. The model suggests that the 1980's was a time-period in which a high federal funds rate existed.

In [ ]:
res_fedfunds.smoothed_marginal_probabilities[1].plot(
    title="Probability of being in the high regime", figsize=(12, 3)
)

From the estimated transition matrix we can calculate the expected duration of a low regime versus a high regime.

In [ ]:
print(res_fedfunds.expected_durations)

A low regime is expected to persist for about fourteen years, whereas the high regime is expected to persist for only about five years.

### Federal funds rate with switching intercept and lagged dependent variable

The second example augments the previous model to include the lagged value of the federal funds rate.

$$r_t = \mu_{S_t} + r_{t-1} \beta_{S_t} + \varepsilon_t \qquad \varepsilon_t \sim N(0, \sigma^2)$$

where $S_t \in \{0, 1\}$, and the regime transitions according to

$$ P(S_t = s_t | S_{t-1} = s_{t-1}) =
\begin{bmatrix}
p_{00} & p_{10} \\
1 - p_{00} & 1 - p_{10}
\end{bmatrix}
$$

We will estimate the parameters of this model by maximum likelihood: $p_{00}, p_{10}, \mu_0, \mu_1, \beta_0, \beta_1, \sigma^2$.

In [ ]:
# Fit the model
mod_fedfunds2 = sm.tsa.MarkovRegression(
    dta_fedfunds.iloc[1:], k_regimes=2, exog=dta_fedfunds.iloc[:-1]
)
res_fedfunds2 = mod_fedfunds2.fit()

In [ ]:
res_fedfunds2.summary()

There are several things to notice from the summary output:

1. The information criteria have decreased substantially, indicating that this model has a better fit than the previous model.
2. The interpretation of the regimes, in terms of the intercept, have switched. Now the first regime has the higher intercept and the second regime has a lower intercept.

Examining the smoothed probabilities of the high regime state, we now see quite a bit more variability.

In [ ]:
res_fedfunds2.smoothed_marginal_probabilities[0].plot(
    title="Probability of being in the high regime", figsize=(12, 3)
)

Finally, the expected durations of each regime have decreased quite a bit.

In [ ]:
print(res_fedfunds2.expected_durations)

### Taylor rule with 2 or 3 regimes

We now include two additional exogenous variables - a measure of the output gap and a measure of inflation - to estimate a switching Taylor-type rule with both 2 and 3 regimes to see which fits the data better.

Because the models can be often difficult to estimate, for the 3-regime model we employ a search over starting parameters to improve results, specifying 20 random search repetitions.

In [ ]:
# Get the additional data
ogap = [-0.53340107, 0.72974336, 2.93532324, 3.58194304, 4.15760183,
        4.28775644, 3.01683831, 2.64185619, 1.82473528, 2.37461162,
        2.39338565, 1.24197006, 1.1370815, -1.28657401, -4.46665335,
        -4.79258966, -3.06711817, -1.3212384, -0.54485309, 0.86588413,
        -0.2469136, -0.75004685, 0.7417022, -0.71350163, -1.5151515,
        -3.80444455, -4.02601957, -3.17873883, -2.48841596, -1.42372882,
        -0.61779928, -0.6430338, -0.73277968, -1.38330388, -1.31537247,
        -0.95626277, 0., -0.15248552, 0.93233085, 1.03888392,
        1.27174389, 0.63400578, 2.13007665, 2.44789481, 3.37605071,
        4.72771597, 6.20753956, 5.39234877, 5.0825758, 4.8605876,
        4.65116262, 3.52755141, 3.35122228, 3.09326482, 4.10191917,
        4.69641066, 4.38452244, 3.79841614, 4.38338947, 3.63766766,
        3.24129653, 1.84967709, 0.75554705, -0.02802691, -0.03673432,
        -1.90527546, -0.14918824, -0.42940569, -0.46382189, -0.97892815,
        -0.12142799, 1.37281513, 1.5143193, 2.47730422, 3.9762032,
        4.08987427, 2.62857127, 2.90107131, 0.97277576, 0.42547619,
        -1.60488391, -2.97784758, -4.98650694, -5.03382635, -4.25698328,
        -3.74993205, -2.39661908, -2.41223454, -2.66694117, -2.62232494,
        -2.29969597, -1.38809109, -0.67855304, -1.08100712, -1.82682908,
        0.92868561, 0.87040615, 1.32669306, 0.56407404, -0.13848817,
        -0.13089494, -0.58975571, -1.00534534, -3.55482054, -4.20365095,
        -2.97225475, -1.57762408, -2.77206445, -2.32418823, -4.01929235,
        -6.25393772, -6.46356869, -7.47437572, -8.06377602, -7.57157278,
        -6.14639282, -5.00167227, -3.74511886, -2.54788184, -1.64858043,
        -1.47994602, -1.44707143, -1.31824112, -1.20102882, -0.57691002,
        -0.64480144, -0.57239723, -0.93083948, -0.8392899, -1.19972074,
        -1.18918467, -0.87174636, -0.78151888, 0.10762761, -0.10596547,
        0.40488175, 0.17958413, 0.67704558, 0.99767941, 1.00495291,
        0.98304421, 0.47067845, 0.80427116, 0.45058677, -0.26300991,
        -1.84629929, -2.99437666, -2.90482664, -3.09490418, -3.32399321,
        -2.87384319, -2.47262239, -2.19618678, -1.91843009, -2.46574545,
        -2.58180451, -2.72212362, -2.17548561, -1.96046102, -1.3287729,
        -1.42521954, -1.04951096, -1.47037697, -1.87099183, -1.72912872,
        -1.76828432, -1.85885167, -0.9193368, -0.95776832, -0.62119246,
        -0.53508854, -0.04090983, 0.47511154, 0.41246772, 0.57928383,
        0.67604625, 1.1378212, 1.96481478, 2.05066752, 1.93714142,
        2.34412026, 3.16807413, 2.57455897, 3.59218717, 2.79711962,
        2.41787243, 1.19362748, 0.82524049, -0.36692095, -1.00542021,
        -0.89346135, -1.23166943, -1.56921482, -2.29188299, -2.56877398,
        -2.37549472, -1.4183135, -1.00017595, -1.03901041, -0.86736482,
        -0.63541794, -0.38296556, 0.11404825, 0.07249562, 0.30608681,
        0.27121997, 0.90333837, 0.595429, 0.08057959, 0.25154814,
        -0.27741581, -0.14053501, -0.06035376, -0.2722317, -1.5122633,
        -1.5272249, -2.5325017, -5.14671373, -6.88223982, -7.36753035,
        -7.43927145, -6.89403868, -6.8306222, -6.26507998, -5.93287086,
        -5.59370756]
inf = [np.nan, np.nan, np.nan, np.nan, -0.2347243,
       0.37373397, 0.25006533, 1.04645514, 2.01665616, 2.58033299,
       3.41399837, 3.60986805, 3.46304512, 3.08529949, 3.45609665,
       3.27347994, 2.29982662, 1.91197193, 0.89083761, 0.390598,
       0.96842253, 1.47531354, 1.39343977, 1.82488036, 1.35991514,
       1.39598227, 1.50695646, 0.8690359, 1.20648873, 0.70517123,
       0.89477205, 1.30740857, 1.20212376, 1.30043352, 1.22895002,
       1.03573787, 1.36272156, 1.39236343, 1.48636675, 1.46398985,
       1.07421875, 1.26611042, 1.1639185, 1.64622331, 1.71658623,
       1.78565705, 2.41930342, 2.6897428, 3.27391338, 3.5685041,
       2.87078357, 2.56671929, 2.70717716, 2.99242783, 3.74010396,
       4.11855173, 4.47761202, 4.62397051, 4.87426901, 5.50198364,
       5.52285719, 5.83354473, 6.22577858, 6.03848171, 5.68597221,
       5.60000038, 4.81102371, 4.31496382, 4.27074528, 3.53535342,
       3.50587225, 3.22580624, 3.02948403, 3.33414626, 4.1129365,
       5.60817289, 6.83709764, 8.41692829, 9.91564655, 10.54788017,
       11.45758915, 12.04798317, 11.13530636, 9.53939915, 8.67963028,
       7.38337183, 6.34047985, 6.01503754, 5.58903217, 5.18573475,
       5.90339899, 6.79609919, 6.57417107, 6.59522104, 6.47466183,
       7.02936935, 8.02397346, 8.9289465, 9.78376389, 10.75433922,
       11.72252846, 12.64148235, 14.20953751, 14.42577076, 12.93487072,
       12.53929329, 11.26111889, 9.87392902, 10.85386753, 9.5831337,
       7.58190918, 6.90676928, 5.81573057, 4.44292784, 3.59408045,
       3.29905081, 2.52680969, 3.23384356, 4.62551022, 4.40519285,
       4.29570436, 4.1543026, 3.64175439, 3.60676312, 3.35249043,
       3.5137701, 3.1053853, 1.67858768, 1.66821122, 1.34587157,
       2.03802228, 3.69979739, 4.16317225, 4.40493536, 3.96511626,
       3.97994113, 4.1420536, 4.3066597, 4.67509222, 5.15961123,
       4.70588255, 4.62759781, 5.23231459, 4.58372736, 5.56420517,
       6.27646685, 5.25958157, 4.84686804, 3.85226536, 2.96485686,
       2.89388347, 3.07301927, 3.07467055, 3.12198234, 3.17306924,
       3.12524581, 2.8174715, 2.76977897, 2.53936958, 2.38237333,
       2.85493255, 2.60332823, 2.84049082, 3.09557867, 2.66420412,
       2.62607908, 2.78390908, 2.8270874, 2.8999064, 3.23162007,
       2.94453382, 2.30179024, 2.22504783, 1.89075232, 1.48277605,
       1.58312511, 1.59639311, 1.5253576, 1.68703699, 2.11280179,
       2.34625125, 2.61982656, 3.25799918, 3.29342604, 3.46889949,
       3.44350553, 3.40975904, 3.32491398, 2.67803454, 1.87507534,
       1.23194993, 1.31765401, 1.57628381, 2.25352097, 2.97640777,
       2.00593972, 2.21688938, 2.00165296, 1.81766617, 2.78586531,
       2.67522621, 3.38513398, 3.0353508, 2.92293549, 3.81956744,
       3.6745038, 3.69086194, 3.92426181, 3.34028482, 1.96539891,
       2.43147993, 2.66511655, 2.34880662, 4.03147316, 4.13719845,
       4.31058264, 5.25250196, 1.59580016, -0.1842365, -0.94229329,
       -1.60695589, 1.48749816, 2.33687115, 1.78588998, 1.22873163,
       1.21550024]
dta_ogap = pd.Series(ogap, index=pd.date_range("1954-07-01", "2010-10-01", freq="QS"))
dta_inf = pd.Series(inf, index=pd.date_range("1954-07-01", "2010-10-01", freq="QS"))

exog = pd.concat((dta_fedfunds.shift(), dta_ogap, dta_inf), axis=1).iloc[4:]

# Fit the 2-regime model
mod_fedfunds3 = sm.tsa.MarkovRegression(dta_fedfunds.iloc[4:], k_regimes=2, exog=exog)
res_fedfunds3 = mod_fedfunds3.fit()

# Fit the 3-regime model
np.random.seed(12345)
mod_fedfunds4 = sm.tsa.MarkovRegression(dta_fedfunds.iloc[4:], k_regimes=3, exog=exog)
res_fedfunds4 = mod_fedfunds4.fit(search_reps=20)

In [ ]:
res_fedfunds3.summary()

In [ ]:
res_fedfunds4.summary()

Due to lower information criteria, we might prefer the 3-state model, with an interpretation of low-, medium-, and high-interest rate regimes. The smoothed probabilities of each regime are plotted below.

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 7))

ax = axes[0]
ax.plot(res_fedfunds4.smoothed_marginal_probabilities[0])
ax.set(title="Smoothed probability of a low-interest rate regime")

ax = axes[1]
ax.plot(res_fedfunds4.smoothed_marginal_probabilities[1])
ax.set(title="Smoothed probability of a medium-interest rate regime")

ax = axes[2]
ax.plot(res_fedfunds4.smoothed_marginal_probabilities[2])
ax.set(title="Smoothed probability of a high-interest rate regime")

fig.tight_layout()

### Switching variances

We can also accommodate switching variances. In particular, we consider the model

$$
y_t = \mu_{S_t} + y_{t-1} \beta_{S_t} + \varepsilon_t \quad \varepsilon_t \sim N(0, \sigma_{S_t}^2)
$$

We use maximum likelihood to estimate the parameters of this model: $p_{00}, p_{10}, \mu_0, \mu_1, \beta_0, \beta_1, \sigma_0^2, \sigma_1^2$.

The application is to absolute returns on stocks, where the data can be found at https://www.stata-press.com/data/r14/snp500.

In [ ]:
# Get the federal funds rate data
areturns = [1.60864139, 0.6581642, 0.91177338,
            1.88970506, 0.76378739, 0.10790635, 0.29509732,
            0.16913767, 1.30772412, 0.85901159, 0.92307973,
            0.9833895, 0.9116146, 2.58575296, 0.36441925,
            1.89720023, 0.65161127, 1.17255056, 0.53518051,
            0.00534112, 1.25064528, 2.00023437, 0.79801333,
            1.42980587, 0.02078664, 2.31948757, 2.78705025,
            1.36003578, 0.15257211, 0.30815724, 0.40030465,
            0.89941251, 0.36925647, 0.75660467, 0.87896836,
            1.07261622, 0.1137321, 1.32838523, 1.03085732,
            1.33930087, 0.66706187, 0.94959277, 1.07173061,
            0.80687243, 1.35347247, 1.56781077, 0.71599048,
            0.50293237, 0.33926481, 2.94415998, 0.72026408,
            0.28967711, 1.05362082, 0.3702977, 2.05277085,
            0.49342933, 0.03423685, 0.34392089, 1.01741159,
            1.43457139, 0.03759775, 1.54626679, 1.07742834,
            0.28664029, 0.72592038, 0.91093767, 0.06915179,
            0.88005662, 0.47802091, 1.2907486, 0.57604247,
            0.71046084, 0.81753206, 0.26241753, 2.57300162,
            0.16590172, 0.2918649, 0.96136051, 1.6711514,
            0.94229084, 1.83614326, 0.28854966, 0.35050908,
            0.04593768, 0.07599987, 0.09888303, 0.12907109,
            2.0099268, 0.23006552, 1.18803704, 0.99970037,
            1.32702613, 0.45646569, 1.43720019, 0.04425191,
            0.53156406, 0.45951003, 1.26583254, 0.26994073,
            0.1238014, 0.53068936, 0.21927625, 0.73882329,
            0.13153869, 0.97837049, 2.36890459, 2.29313374,
            0.75562358, 0.08656374, 2.4979558, 0.64189923,
            0.22916116, 2.27840376, 0.46641645, 2.02508688,
            1.25530422, 1.27711689, 0.07773363, 0.23380435,
            1.58663058, 0.19108967, 0.52218717, 0.18055375,
            1.18262017, 0.47418493, 0.88282752, 0.98944044,
            1.04560554, 0.65470523, 0.2604697, 0.14658713,
            0.77688956, 1.10911596, 0.69967973, 1.04578161,
            0.29641318, 0.98087156, 0.46531865, 0.11846001,
            0.44440377, 1.11066306, 0.02238905, 0.19865835,
            1.48028743, 0.27695858, 0.9391492, 1.70575404,
            2.94507742, 0.35386264, 0.72816408, 1.80369282,
            0.12440593, 1.04197288, 1.2957871, 1.35031664,
            0.55384284, 1.13915396, 0.29186234, 1.21344364,
            0.23005128, 0.85578758, 1.80613887, 1.55996382,
            1.46395147, 0.59826899, 0.65880769, 1.68974137,
            1.12778795, 4.19566727, 0.14379959, 2.09945345,
            0.29264972, 1.25936544, 0.84738803, 0.54094779,
            2.27655816, 1.48392296, 1.13808954, 1.16038692,
            0.46204364, 2.09433556, 1.16782069, 2.0192802,
            2.6190269, 1.63471925, 0.25279006, 2.64083171,
            1.64290273, 2.42852569, 1.54714262, 1.14975035,
            3.59362221, 1.16689992, 5.11030865, 1.81326246,
            0.93489766, 1.38605726, 0.53841805, 1.02298951,
            2.03038621, 2.8340385, 0.13691254, 3.18769765,
            0.23076122, 1.95332313, 1.63122225, 2.66484141,
            0.86377442, 1.1782372, 0.57231718, 1.11979997,
            2.07001758, 0.08726255, 1.71130466, 1.04979181,
            1.9825747, 3.43235064, 1.50204682, 1.75699294,
            2.56816769, 0.75786251, 0.93131924, 1.45494628,
            0.49975556, 0.32756457, 0.47183469, 3.3737793,
            2.25759649, 0.34138981, 3.09048033, 10.32189178,
            10.15319347, 0.12398402, 4.65263939, 7.62032652,
            7.04052448, 4.55579329, 3.52704573, 3.38968754,
            3.00466204, 0.46617937, 1.42808878, 1.00660408,
            4.65142584, 5.20996618, 4.80301046, 0.99780792,
            1.15280604, 1.87296033, 4.60985804, 5.41294718,
            6.06733084, 3.18375754, 10.0548315, 4.22182512,
            1.24640226, 2.66358495, 2.60049844, 0.00352026,
            1.02208447, 4.09924603, 1.27764511, 0.90124834,
            0.5303241, 3.84383249, 1.24640775, 1.39796948,
            2.34609175, 1.7742399, 3.56689548, 1.27681601,
            5.32056713, 3.19770503, 1.89575887, 0.59274858,
            0.64010525, 2.65920091, 0.81912726, 0.4868626,
            3.13063931, 1.3960743, 1.03451502, 1.28983963,
            3.27489519, 1.41772103, 2.00014663, 2.02787399,
            3.50289273, 1.65296888, 0.02450024, 0.04084374,
            0.17252181, 0.78132814, 0.20216605, 1.48436368,
            0.3301619, 1.12080252, 0.00699845, 3.87074757,
            0.84627002, 2.26680374, 2.07992935, 1.62452054,
            0.66078293, 2.26608515, 1.58540344, 0.98763937,
            0.25370923, 1.2576412, 1.07146478, 0.48786601,
            0.02327727, 1.29385257, 3.52217674, 1.05305433,
            5.13598871, 1.43351507, 2.12951326, 3.03700447,
            0.65843326, 4.28524971, 2.3428576, 4.72853422,
            0.58606911, 2.70345545, 0.8207835, 0.16228235,
            2.80714321, 1.97183621, 0.5928334, 3.61601782,
            1.82700455, 1.52638936, 0.72525144, 0.6499536,
            1.58741212, 0.72647524, 0.65064299, 0.43771812,
            2.68048692, 2.20902133, 0.0988697, 0.31138307,
            2.79860616, 1.13209391, 0.91427463, 0.69550049,
            0.68990183, 0.65359998, 1.04932129, 0.00310441,
            0.48663121, 1.68144464, 0.99051267, 0.22263506,
            0.97846323, 0.55040002, 2.56734443, 0.12510587,
            2.15363359, 1.18440747, 0.66974002, 0.48981813,
            2.08285856, 1.03952742, 1.00747502, 0.52523118,
            0.81593889, 0.22168602, 2.73786068, 1.21678591,
            0.235705, 0.56248677, 3.66057348, 0.35822684,
            0.97550339, 1.21677041, 4.03415823, 9.10342026,
            2.24355674, 3.6120553, 4.36456299, 0.83891636,
            1.07712805, 2.28685427, 4.04548168, 1.67408013,
            4.57762337, 2.47123241, 1.88890803, 1.62245703,
            0.02149973, 0.48483402, 4.40716505, 0.28621164,
            4.56798553, 1.6255945, 0.6124717, 2.72943926,
            0.80645156, 1.26738918, 0.91451788, 1.59959269,
            0.0356785, 1.93719864, 0.42164543, 0.87313241,
            0.52508104, 0.44771862, 1.38226497, 1.83891225,
            0.00711749, 0.26621303, 2.25254321, 0.27307722,
            0.26436633, 1.80608702, 2.29477572, 2.0931437,
            2.2915051, 0.82041657, 2.09074521, 1.87793779,
            2.15142703, 1.549685, 2.44940472, 0.45297864,
            0.35515305, 0.23224437, 1.77138305, 0.98827285,
            0.98435384, 0.80031335, 0.49445853, 0.36061874,
            2.15444446, 1.92558503, 0.75404048, 0.31921348,
            0.32092738, 0.48054051, 0.98650485, 1.1810472,
            0.28533801, 3.02953291, 0.16818592, 2.20164418,
            0.3911584, 0.6942575, 0.55016953, 0.06157291,
            0.19509397, 2.3744297, 0.73775989, 1.12842739,
            0.87197775, 0.30168825, 0.71310955, 0.27689508,
            1.13476491, 1.60331428, 1.56165123, 0.31513214,
            0.02698154, 0.49029687, 0.17265303, 0.36386153,
            0.56225872, 1.59077382, 1.84919345, 1.4230696,
            1.28607559, 0.57890779, 1.14760947, 0.22594096,
            0.43510813, 2.90668917, 1.49716794, 1.9549973,
            2.10786223, 0.71948445, 0.19396119, 0.86563414,
            0.63498968, 2.3593328, 0.18950517, 0.45737442,
            1.82937241, 1.72589195, 0.29414186, 0.74434268,
            1.22564518, 2.01444268, 2.32068515, 0.98414028,
            0.1174908, 0.22450124, 1.24669802, 0.70953292,
            0.21857196, 0.11119327, 0.60500813, 2.04446197,
            1.146896, 0.54849964, 0.23402978, 0.32219616,
            2.7076292, 1.57800817, 2.08260155, 1.81090641,
            0.45189673, 1.01260054, 0.65379494, 0.94736898,
            0.37556711, 0.44287458, 0.34578958, 1.48449266,
            1.95924711, 0.09717447]

dta_areturns = pd.Series(
    areturns, index=pd.date_range("2004-05-04", "2014-5-03", freq="W")
)

# Plot the data
dta_areturns.plot(title="Absolute returns, S&P500", figsize=(12, 3))

# Fit the model
mod_areturns = sm.tsa.MarkovRegression(
    dta_areturns.iloc[1:],
    k_regimes=2,
    exog=dta_areturns.iloc[:-1],
    switching_variance=True,
)
res_areturns = mod_areturns.fit()

In [ ]:
res_areturns.summary()

The first regime is a low-variance regime and the second regime is a high-variance regime. Below we plot the probabilities of being in the low-variance regime. Between 2008 and 2012 there does not appear to be a clear indication of one regime guiding the economy.

In [ ]:
res_areturns.smoothed_marginal_probabilities[0].plot(
    title="Probability of being in a low-variance regime", figsize=(12, 3)
)